# Script for files WITHOUT pre-defined phase values


1. Import Libraries


In [ ]:
# Import Libraries
from google.colab import drive
import numpy as np

2. Connect to GDrive if needed

In [ ]:
# Connect to GDrive if the files are available on the GDrive
# drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


3. Helper Function 1 - <br>
To create structured data from the file

In [ ]:
# Helper Function - create structured data from the file

def get_data(j,pair_paths,data,max_pairs):
    result = []
    for i in range(pair_paths):
        line = data[j].decode('utf-8').strip().split()

        Tx = data[j+2].decode('utf-8').strip().split()
        Tx = [float(i) for i in Tx]
        j = j+int(line[1]) + 4
        power = float(line[2])
        aoa = float(line[3])
        diff = int(line[1])
        Rx = data[j-1].decode('utf-8').strip().split()

        Rx = [float(i) for i in Rx]
        tmp = [Tx,Rx,[power,aoa,diff]]
        result.append(tmp)
    # print(Tx, Rx, [power, aoa, diff])
    result = np.array(result)
    return j,result

4. Helper Function 2 - <br>
To get Power and Delay lists from the structured data

In [ ]:
# Helper Function - get Power and Delay lists from the structured data

def getPowerDelay(rawData):
        power = []
        delay = []
        for i in range(len(rawData)):
          tmp = []
          d = []
          if rawData[i].size != 0:                             #change identation of power.append and change pos of tmp = [] to make len less than 29929
            for j in range(len(rawData[i])):
              tmp.append(rawData[i][j][2][0])
              d.append(rawData[i][j][2][1])
          power.append(np.array(tmp))
          delay.append(np.array(d))
        return power,delay

5. Helper Function 3 - <br>
To get all Rx and Tx values from the structured data

In [ ]:
# Helper Function - get all Rx and Tx values from the structured data

def getRxTx(rawData):
        Rx_position = []
        Tx_position = []
        for i in range(len(rawData)):
          tmp = []
          d = []
          if rawData[i].size != 0:
            for j in range(len(rawData[i])):
              tmp.append(rawData[i][j][0])
              d.append(rawData[i][j][1])
          Rx_position.append(np.array(d))
          Tx_position.append(np.array(tmp))
        return Tx_position,Rx_position

6. Helper Function 4 - <br>
To remove data with empty values

In [ ]:
# Helper Function - remove data with 0 values

def removeEmpty(l):
  filtered_list = [arr for arr in l if arr.size > 0]
  return filtered_list

7. Helper Function 5 - <br>
To check if the Rx and Tx values are the same throughout the data

In [ ]:
# Helper Function - check if the Rx and Tx values are the same throughout the data

def checkArray(l):
  indices = {}
  for i in range(len(l)):
   comparison_results = [np.array_equal(arr, l[i][0]) for arr in l[i]]
   for j in range(len(comparison_results)):
    if comparison_results[j] == False:
      indices[i] = j
  return indices

8. Helper Function 6 - <br>
To generate the Phase values



In [ ]:
# Helper Function - generate the Phase values

def generatePhase(size):
  theta = np.random.uniform(0, np.pi, (size,))
  return theta

#Main Function 1<br>
To get raw Rx, Tx, Power and delay values from the file

In [ ]:
# Main Function 1 - returns raw Rx, Tx, Power and delay values from the file

def getData(file):
    final = []
    with open(file, 'rb') as file:
      data = file.readlines()
      total_pairs = int(data[1].decode('utf-8').strip())
      pair_paths = int(data[2].decode('utf-8').strip().split()[1])
      j = 4
      j,result = get_data(j,pair_paths,data,pair_paths)
      final.append(result)

      for i in range(total_pairs - 1):
        pair_paths = int(data[j].decode('utf-8').strip().split()[1])

        if pair_paths == 0:
          j,result = get_data(j + 1,pair_paths,data,pair_paths)
          final.append(result)
        else:
          j,result = get_data(j + 2,pair_paths,data,pair_paths)
          final.append(result)
    Tx_position = []
    Rx_position = []
    power = []
    delay = []
    Tx_position, Rx_position = getRxTx(final)
    power, delay = getPowerDelay(final)
    return Tx_position, Rx_position, power, delay

#Main Function 2<br>
1. To remove repetetions of data and checks whether all values are same throught the data
2. Returns - Tx, Rx, Power, Delay arrays

In [ ]:
# Main Function 2 - Remove repetetions of data and checks whether all values are same throught the data
# Returns - Tx, Rx, Power, Delay arrays
# Tx - 3 x 1
# Rx - 27099 x 1
# Power - 27099 x 1
# Delay - 27099 x 1

def getDataWitoutEmptyRepeat(Tx_position, Rx_position, power, delay):
  Tx_position = removeEmpty(Tx_position)
  Rx_position = removeEmpty(Rx_position)
  power = removeEmpty(power)
  delay= removeEmpty(delay)
  indices = checkArray(Tx_position)

  if indices == {}:
    Tx = Tx_position[0][0]

  indices = checkArray(Rx_position)

  if indices == {}:
    Rx = []
    for i in range(len(Rx_position)):
      Rx.append(Rx_position[i][0])

  return Tx, Rx, power, delay

# Main Function 3<br>
To Calculate
 Channel Frequency Values


In [ ]:
# Main Function 3 - Get Channel Frequency Values

def getDataChannelFrequency(power, delay, M, bW):
  channelFreq = []
  for i in range(len(power)):
    theta = generatePhase(len(power[i]))
    newPower = 10 ** ((power[i] - 30) / 20)
    comp = np.zeros((len(power[i]), ), dtype=np.complex128)
    comp = np.exp(1j * theta)
    newPower = newPower * comp

    freq = np.linspace(-bW/2, bW/2 - bW/M, M)
    tmp = []
    for j in range(len(freq)):
      h = np.exp(1j * 2 * np.pi * freq[j] * delay[i])
      a = newPower * h
      tmp.append(sum(a))


    channelFreq.append(tmp)
  channelFreq = np.array(channelFreq)
  return channelFreq

# Main Function 4 <br>
To calculate Channel Frequency Values directly from the file with raw data

In [ ]:
# Main Function 4 - Get Channel Frequency by adding the filename directly in the arguments
# Channel Frequency Version 2

def getDataChannelFrequencyFileName(file, M, bW):
    final = []
    with open(file, 'rb') as file:
      data = file.readlines()
      total_pairs = int(data[1].decode('utf-8').strip())
      pair_paths = int(data[2].decode('utf-8').strip().split()[1])
      j = 4
      j,result = get_data(j,pair_paths,data,pair_paths)
      final.append(result)

      for i in range(total_pairs - 1):
        pair_paths = int(data[j].decode('utf-8').strip().split()[1])

        if pair_paths == 0:
          j,result = get_data(j + 1,pair_paths,data,pair_paths)
          final.append(result)
        else:
          j,result = get_data(j + 2,pair_paths,data,pair_paths)
          final.append(result)
    Tx_position = []
    Rx_position = []
    power = []
    delay = []
    Tx_position, Rx_position = getRxTx(final)
    power, delay = getPowerDelay(final)

    Tx_position = removeEmpty(Tx_position)
    Rx_position = removeEmpty(Rx_position)
    power = removeEmpty(power)
    delay= removeEmpty(delay)

    # print(len(power))

    indices = checkArray(Tx_position)

    if indices == {}:
      Tx = Tx_position[0][0]

    indices = checkArray(Rx_position)

    if indices == {}:
      Rx = []
      for i in range(len(Rx_position)):
        Rx.append(Rx_position[i][0])

    channelFreq = []
    for i in range(len(power)):
      theta = generatePhase(len(power[i]))
      newPower = 10 ** ((power[i] - 30) / 20)
      comp = np.zeros((len(power[i]), ), dtype=np.complex128)
      comp = np.exp(1j * theta)
      newPower = newPower * comp

      freq = np.linspace(-bW/2, bW/2 - bW/M, M)
      tmp = []
      for j in range(len(freq)):
        h = np.exp(1j * 2 * np.pi * freq[j] * delay[i])
        a = newPower * h
        tmp.append(sum(a))


      channelFreq.append(tmp)
    channelFreq = np.array(channelFreq)
    return channelFreq

# Main Function 5 <br>
To calculate Channel Frequency of data with empty values

In [ ]:
# Main Function 5 - Get Channel Frequency of data with empty values.
# Channel Frequency Version 3

def getDataChannelFreqWithEmpty(Tx_position, Rx_position, power, delay, M, bW):
    indices = checkArray(Tx_position)

    if indices == {}:
      Tx = Tx_position[0][0]

    indices = checkArray(Rx_position)

    if indices == {}:
      Rx = []
      for i in range(len(Rx_position)):
        if Rx_position[i].size != 0:
          Rx.append(Rx_position[i][0])
        else:
          Rx.append([])
    channelFreq = []
    for i in range(len(power)):
      theta = generatePhase(len(power[i]))
      newPower = 10 ** ((power[i] - 30) / 20)
      comp = np.zeros((len(power[i]), ), dtype=np.complex128)
      comp = np.exp(1j * theta)
      newPower = newPower * comp

      freq = np.linspace(-bW/2, bW/2 - bW/M, M)
      tmp = []
      for j in range(len(freq)):
        h = np.exp(1j * 2 * np.pi * freq[j] * delay[i])
        a = newPower * h
        tmp.append(sum(a))


      channelFreq.append(tmp)
    channelFreq = np.array(channelFreq)
    return channelFreq

In [ ]:
fileName = ''   # Insert file path of .p2m file

In [ ]:
Tx_position, Rx_position, power, delay = getData(fileName)

In [ ]:
Tx, Rx, power, delay = getDataWitoutEmptyRepeat(Tx_position, Rx_position, power, delay)

In [ ]:
# Parameters for channel frequency calculations

M = 200
bW = 400

In [ ]:
channelFreq = getDataChannelFrequency(power, delay, M, bW)

In [ ]:
channelFreq = getDataChannelFreqWithEmpty(Tx_position, Rx_position, power, delay, M, bW)